In [113]:
import re
import numpy as np
import pandas as pd
import kss

### 카카오 리뷰 데이터 문장 단위로 끊어서 카테고리 추정

In [100]:
kakao = pd.read_csv("../../datas/kakao_review_cat_predict.csv")
kakao = kakao[['contents', 'place_name', 'point', 'username', 'date']]
# 후기 없이 별점만 있는 데이터 drop
kakao.drop(kakao[kakao['contents'].isnull()==True].index, axis=0, inplace=True)
kakao['contents'] = [r.replace("/n", ". ") for r in kakao['contents']]
kakao['contents'] = [re.sub(r"[^가-힣0-9.,'?!]", " ", r) for r in kakao['contents']]
# 문장별 분리
kakao['contents_sent'] = [kss.split_sentences(r) for r in kakao['contents']]
kakao = kakao.reset_index(drop=True)
kakao

,contents,place_name,point,username,date,contents_sent
0,정말 무릉도원 같은곳 넘 좋아서 한번가보고 다시 또 감 또 재방문할꺼임,산골캠핑장,5,김정희,2020.08.09.,[정말 무릉도원 같은곳 넘 좋아서 한번가보고 다시 또 감 또 재방문할꺼임]
1,샤워시설도 깔끔하고 아주 좋아요. 뒤에 망봉산도 높은 산이 아니고 가족들 산책하기 ...,궁농오토캠핑장,5,깜~♡,2020.05.27.,"[샤워시설도 깔끔하고 아주 좋아요., 뒤에 망봉산도 높은 산이 아니고 가족들 산책하..."
2,좋습니다.,강화캐라반캠핑장,5,열,2021.04.27.,[좋습니다.]
3,항상 잘 이용하고있읍니다,강화캐라반캠핑장,5,고바우,2020.01.26.,[항상 잘 이용하고있읍니다]
4,"가격이 저렴한데 비해 깔끔하고 좋았던거 같아요, 물 적게 나오는거 외에는 불편함은 ...",강화캐라반캠핑장,4,자두님,2019.07.30.,"[가격이 저렴한데 비해 깔끔하고 좋았던거 같아요,, 물 적게 나오는거 외에는 불편함..."
...,...,...,...,...,...,...
3709,생긴지얼마안되서 깨끗하긴하지만 시에서 운영하는것도 아닌게 사만원이나 받으면서 매점은...,양양오색 휴랜드 야영장,2,girl,2020.12.20.,[생긴지얼마안되서 깨끗하긴하지만 시에서 운영하는것도 아닌게 사만원이나 받으면서 매점...
3710,카페 위치랑 쉬기딱좋은곳.. 테이블 의자셋팅 풍경 너무좋네요.. 하지만 커피맛은 보...,귀촌 관광농원,3,배현환,2021.04.24.,"[카페 위치랑 쉬기딱좋은곳.. 테이블 의자셋팅 풍경 너무좋네요.., 하지만 커피맛은..."
3711,2층 테라스도 있고 주차장 넓어서 좋아요 정원도 있어서 아이들도 놀기 좋네요,귀촌 관광농원,5,심석연(바라봄의법칙),2021.03.23.,"[2층 테라스도 있고 주차장 넓어서 좋아요, 정원도 있어서 아이들도 놀기 좋네요]"
3712,넓고 한적하고 정원 예쁘고 테라스까지 넘 좋아요,귀촌 관광농원,5,로지,2021.03.20.,[넓고 한적하고 정원 예쁘고 테라스까지 넘 좋아요]


In [101]:
# username이 null값이 조금 있긴한데 그냥 패스
kakao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3714 entries, 0 to 3713
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   contents       3714 non-null   object
 1   place_name     3714 non-null   object
 2   point          3714 non-null   int64 
 3   username       3278 non-null   object
 4   date           3714 non-null   object
 5   contents_sent  3714 non-null   object
dtypes: int64(1), object(5)
memory usage: 174.2+ KB


In [102]:
# 캠핑장별 평균 별점 -> 네이버의 유저별 평균별점 대신 사용
camp_star = kakao.groupby('place_name').mean().round(3)
camp_star

,point
place_name,
(주)밀모 빨강치마 야영장,4.500
(주)쉐르빌리안티티,4.667
(주)아웃오브파크,3.750
(주)어반슬로우시티,4.000
(주)태평소금 천일염 힐링캠프,5.000
...,...
휴파크 오토캠핑장,1.000
흙집섬진강풍경,1.000
힐링가평오토캠핑장,5.000


In [104]:
sep_sent_list, place_name_list, point_list, username_list, date_list = [], [], [], [], []

for idx, cs in enumerate(kakao['contents_sent']):
    for sent in cs:
        sep_sent_list.append(sent)
        place_name_list.append(kakao['place_name'].iloc[idx])
        point_list.append(kakao['point'].iloc[idx])
        username_list.append(kakao['username'].iloc[idx])
        date_list.append(kakao['date'].iloc[idx])
        
new_kakao = pd.DataFrame({'review': sep_sent_list, 
                         'camp': place_name_list, 
                          'point': point_list,
                          'username': username_list,
                          'date': date_list})
new_kakao

,review,camp,point,username,date
0,정말 무릉도원 같은곳 넘 좋아서 한번가보고 다시 또 감 또 재방문할꺼임,산골캠핑장,5,김정희,2020.08.09.
1,샤워시설도 깔끔하고 아주 좋아요.,궁농오토캠핑장,5,깜~♡,2020.05.27.
2,뒤에 망봉산도 높은 산이 아니고 가족들 산책하기 좋은 올레길느낌 초입때 경사가 ...,궁농오토캠핑장,5,깜~♡,2020.05.27.
3,꼭 가볼만한 곳 강츄입니다,궁농오토캠핑장,5,깜~♡,2020.05.27.
4,좋습니다.,강화캐라반캠핑장,5,열,2021.04.27.
...,...,...,...,...,...
8880,2층 테라스도 있고 주차장 넓어서 좋아요,귀촌 관광농원,5,심석연(바라봄의법칙),2021.03.23.
8881,정원도 있어서 아이들도 놀기 좋네요,귀촌 관광농원,5,심석연(바라봄의법칙),2021.03.23.
8882,넓고 한적하고 정원 예쁘고 테라스까지 넘 좋아요,귀촌 관광농원,5,로지,2021.03.20.
8883,한적하고 고즈넉해요.,귀촌 관광농원,3,꽃밭에는 꽃들이,2021.03.02.


In [81]:
new_kakao.to_csv("../../datas/kakao_review_sent.csv")

#### naver카테고리 분류기로 위 데이터프레임 카테고리 predict 진행

In [112]:
# predict 완료된 파일 불러오기
cat_kakao = pd.read_csv("../../datas/model2_cat_predict.csv")
cat_kakao = cat_kakao[['camp', 'review', 'point','category']]
cat_kakao['avg_point'] = [camp_star['point'][name] for name in cat_kakao['camp']]
cat_kakao

,camp,review,point,category,avg_point
0,산골캠핑장,정말 무릉도원 같은곳 넘 좋아서 한번가보고 다시 또 감 또 재방문할꺼임,5.0,만족도,5.00
1,궁농오토캠핑장,샤워시설도 깔끔하고 아주 좋아요.,5.0,부대/공용시설,5.00
2,궁농오토캠핑장,뒤에 망봉산도 높은 산이 아니고 가족들 산책하기 좋은 올레길느낌 초입때 경사가 ...,5.0,분위기,5.00
3,궁농오토캠핑장,꼭 가볼만한 곳 강츄입니다,5.0,메인시설,5.00
4,강화캐라반캠핑장,좋습니다.,5.0,메인시설,4.75
...,...,...,...,...,...
8880,귀촌 관광농원,2층 테라스도 있고 주차장 넓어서 좋아요,5.0,부대/공용시설,4.00
8881,귀촌 관광농원,정원도 있어서 아이들도 놀기 좋네요,5.0,부대/공용시설,4.00
8882,귀촌 관광농원,넓고 한적하고 정원 예쁘고 테라스까지 넘 좋아요,5.0,부대/공용시설,4.00
8883,귀촌 관광농원,한적하고 고즈넉해요.,3.0,메인시설,4.00


In [114]:
cat_kakao.to_csv("../../datas/kakao_review_cat_revised.csv")